In [73]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import pickle
from sklearn.metrics import accuracy_score  
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [2]:
df = pd.read_csv(r"C:\Users\rahul\Downloads\train.txt", sep=";", header=None, names=['text', 'sentiment'])
df.head(2)

,text,sentiment
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness


In [3]:
df.isnull().sum()

text         0
sentiment    0
dtype: int64

In [4]:
df.isna().sum()

text         0
sentiment    0
dtype: int64

In [5]:
df.sentiment.value_counts()

sentiment
joy         5362
sadness     4666
anger       2159
fear        1937
love        1304
surprise     572
Name: count, dtype: int64

In [6]:
def change_sentiment(df):
    # Group 'joy', 'love', and 'surprise' into 'positive'
    df.loc[df['sentiment'].isin(['joy', 'love', 'surprise']), 'sentiment'] = 'positive'

    # Group 'anger', 'fear', and 'sadness' into 'negative'
    df.loc[df['sentiment'].isin(['anger', 'fear', 'sadness']), 'sentiment'] = 'negative'

    return df
df = change_sentiment(df)

In [7]:
print(df.text	.iloc[155])
print(df.sentiment.iloc[155])

i could feel her whimper to the thought of being unloved and uncared for
negative


In [8]:
def clean_text(text):
  return text.translate(str.maketrans('', '', string.punctuation))

In [9]:
df["text"]=df.text.apply(clean_text)
df.head()

,text,sentiment
0,i didnt feel humiliated,negative
1,i can go from feeling so hopeless to so damned...,negative
2,im grabbing a minute to post i feel greedy wrong,negative
3,i am ever feeling nostalgic about the fireplac...,positive
4,i am feeling grouchy,negative


In [10]:
def remove_number(txt):
  new = ""
  for i in txt:
      if not i.isdigit():
          new= new + i
  return new
df["text"]=df.text.apply(clean_text)

In [11]:
def remove_number(txt):
  new = ""
  for i in txt:
      if not i.isaccii():
          new= new + i
  return new
df["text"]=df.text.apply(clean_text)

In [123]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rahul\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rahul\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
stop_words = set(stopwords.words('english'))

In [15]:
def remove_stopwords(txt):
    words = word_tokenize(txt)
    cleaned = []
    for i in words:
        if not i in stop_words:
            cleaned.append(i)
    return" ".join(cleaned)
df["text"] = df["text"].apply(remove_stopwords)


In [16]:
def remove_noisy(txt):
    words = word_tokenize(txt)
    cleaned = []
    for i in words:
        if not re.search(r'(.)\1{2,}', i) and not re.search(r'\W', i):
            cleaned.append(i)
    return" ".join(cleaned)
df.text = df.text.apply(remove_noisy)

In [20]:
from nltk.stem import PorterStemmer

ps = PorterStemmer()
def stem_words(txt):
    words = word_tokenize(txt)
    stemmed = []
    for i in words:
        stemmed.append(ps.stem(i))
    return" ".join(stemmed)
df.text = df.text.apply(stem_words)

In [25]:
df.sentiment.value_counts()

sentiment
negative    8762
positive    7238
Name: count, dtype: int64

In [ ]:
df.sentiment = df.sentiment.replace({"negative": 0, "positive": 1})

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['sentiment'], test_size=0.2, random_state=42)


In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)


In [43]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=1000)
model.fit(X.toarray(), y_train)

LogisticRegression(max_iter=1000)

In [45]:
y_pred = model.predict(X_test_vectorized.toarray())
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.93

In [72]:
list = "Spending time with you feels like sunshine on a rainy day. You make everything better, just by being there."
list = vectorizer.transform([list])
list = list.toarray()
p= model.predict(list)[0]

In [74]:
pickle.dump(model, open('model.pkl', 'wb'))
pickle.dump(vectorizer, open('vectorizer.pkl', 'wb'))

In [ ]:
def text(text):
    text = str.lower(text)
    clean = text.translate(str.maketrans('', '', string.punctuation))
    words = word_tokenize(clean)
    filtered_words = [w for w in words if w not in stop_words]
    wod = [w for w in filtered_words if not re.search(r'(.)\1{2,}', w) and not re.search(r'\W', w)]
    stemmed = [ps.stem(w) for w in wod]
    # Remove digit-only and non-ASCII words
    filtered = [w for w in stemmed if not w.isdigit() and w.isascii()]
    return " ".join(filtered)

In [121]:
list = "This product is soooo 88 badddd!!! It broke like evveryy time I try to use it. Weeeeell, totally not worth it."
gg = text(list)

In [122]:
gg

'product broke like evveryy time tri use total worth'